In [1]:
import datetime
import re
import os

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

In [2]:
import json
from datetime import datetime, timezone, timedelta
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

In [3]:
def retrive_calendar_events(user, start, end):
    events_list = []
    token_path = "Keys/"+user.split("@")[0]+".token"
    user_creds = Credentials.from_authorized_user_file(token_path)
    calendar_service = build("calendar", "v3", credentials=user_creds)
    events_result = calendar_service.events().list(calendarId='primary', timeMin=start,timeMax=end,singleEvents=True,orderBy='startTime').execute()
    events = events_result.get('items')

    count=0
    for event in events : 
        attendee_list = []
        try:
            for attendee in event["attendees"]: 
                attendee_list.append(attendee['email'])
        except: 
            attendee_list.append("SELF")
        try:
            start_time = event["start"]["dateTime"]
            end_time = event["end"]["dateTime"]
            events_list.append(
                {"StartTime" : start_time, 
                 "EndTime": end_time, 
                 "NumAttendees" :len(set(attendee_list)), 
                 "Attendees" : list(set(attendee_list)),
                 "Summary" : event["summary"]})
        except Exception as E:
            count+=1
    print('No of exceptions are: ',count)
    return events_list

In [4]:
event = retrive_calendar_events("userone.amd@gmail.com", '2023-07-17T00:00:00+05:30', '2026-07-17T23:59:59+05:30')

No of exceptions are:  52


In [5]:
event[0]

{'StartTime': '2025-06-29T18:00:00+05:30',
 'EndTime': '2025-06-30T09:00:00+05:30',
 'NumAttendees': 1,
 'Attendees': ['SELF'],
 'Summary': 'Off Hours'}

In [6]:
os.environ["LANGSMITH_API_KEY"] = "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"
os.environ["LANGSMITH_TRACING"] = "true"

In [7]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from datetime import datetime
from langchain_core.output_parsers import JsonOutputParser

def extract_meeting_details_with_llm(email_content, request_datetime_str):

    reference_datetime = datetime.strptime("19-07-2025T12:34:55", "%d-%m-%YT%H:%M:%S")
    day = reference_datetime.weekday()
    
    # The system prompt now includes few-shot examples
    system_prompt = """You are an expert meeting information extraction assistant. Your task is to analyze email content and extract meeting details with precise date and time information.

## Core Instructions

1. **Extract, don't schedule**: Your role is to extract meeting information from the provided text, not to suggest or schedule meetings.

2. **Date/Time Resolution**: Use the provided reference datetime to resolve relative expressions like "tomorrow", "next week", or specific days of the week.

3. **Time Defaults**: 
   - When no specific start time is mentioned, use 00:00:00 (start of day)
   - When no specific end time is mentioned, use 23:59:59 (end of day)
   - For same-day/week/month meetings without specific time (e.g., "this week"), use current time as start

4. **Duration Calculation**:
   - If both start/end times and duration are specified, prioritize the time range for dates
   - Calculate duration from the actual start/end time difference
   - If only duration is given, calculate end time from start time + duration

5. **Future Dates Only**: All extracted dates must be in the future relative to the current datetime.

## Output Format

Return ONLY a valid JSON object with exactly these three keys:

{{
    "chain_of_thought": "Your detailed reasoning for identifying dates, times, and duration calculations",
    "start_date": "Start datetime in 'dd-mm-YYYYTHH:MM:SS' format or null if not determinable",
    "end_date": "End datetime in 'dd-mm-YYYYTHH:MM:SS' format or null if not determinable", 
    "duration_minutes": "Total meeting duration in minutes (integer) or null if not determinable"
}}

## Examples

### Example 1: Basic relative date with duration
**Current Datetime:** 19-07-2025T14:30:00
**Current Day:** Friday
**Email:** "Let's schedule our project review meeting tomorrow for 1 hour."

Output:
{{
    "chain_of_thought": "Current date: 19-07-2025 (Friday). 'Tomorrow' = current date + 1 day = 20-07-2025 (Saturday). No specific time mentioned, so applying defaults: start = 00:00:00, end = 23:59:59. Duration explicitly stated as 1 hour = 60 minutes.",
    "start_date": "20-07-2025T00:00:00",
    "end_date": "20-07-2025T23:59:59",
    "duration_minutes": 60
}}

### Example 2: Conflicting duration and time range
**Current Datetime:** 02-12-2025T16:45:30
**Current Day:** Tuesday
**Email:** "Board meeting Thursday 2-5 PM for 2 hours discussion."

Output:
{{
    "chain_of_thought": "Current date: 02-12-2025 (Tuesday). 'Thursday' = find next Thursday = 02-12-2025 + 2 days = 04-12-2025. Time conversion: 2 PM = 14:00:00, 5 PM = 17:00:00. Duration from time range: 17:00 - 14:00 = 3 hours = 3 × 60 = 180 minutes. Email mentions '2 hours' but explicit time range takes priority, so actual duration = 180 minutes.",
    "start_date": "04-12-2025T14:00:00",
    "end_date": "04-12-2025T17:00:00",
    "duration_minutes": 180
}}

### Example 3: Specific date with time
**Current Datetime:** 28-11-2025T11:20:15
**Current Day:** Friday
**Email:** "Client presentation on December 3rd after 3:30 PM for 90 minutes."

Output:
{{
    "chain_of_thought": "Current date: 28-11-2025. 'December 3rd' = 03-12-2025 (future date confirmed: 03-12 > 28-11). Time conversion: 3:30 PM = 15:30:00. Duration: 90 minutes. No specific end time mentioned, so applying defaults: end = 23:59:59",
    "start_date": "03-12-2025T15:30:00",
    "end_date": "03-12-2025T23:59:59",
    "duration_minutes": 90
}}

### Example 4: Same day meeting
**Current Datetime:** 10-04-2025T09:45:30
**Current Day:** Thursday
**Email:** "Quick sync meeting within today for 30 minutes."

Output:
{{
    "chain_of_thought": "Current datetime: 10-04-2025T09:45:30 (Thursday). 'Within today' = same day but no specific time mentioned, so start from current time = 09:45:30. End time for same-day meetings without specific time = end of day = 23:59:59. Duration specified: 30 minutes.",
    "start_date": "10-04-2025T09:45:30",
    "end_date": "10-04-2025T23:59:59",
    "duration_minutes": 30
}}

### Example 5: Flexible weekly meeting
**Current Datetime:** 18-09-2025T16:30:00
**Current Day:** Thursday
**Email:** "Training session any time next week for 2 hours."

Output:
{{
    "chain_of_thought": "Current date: 18-09-2025 (Thursday). 'Next week' = business week starting from next Monday. Next Monday = 18-09-2025 + 4 days = 22-09-2025. Since 'any time' is mentioned with no specific time, using default: start = 22-09-2025T00:00:00. End of business week = Friday = 22-09-2025 + 4 days = 26-09-2025T23:59:59. Duration specified: 2 hours = 2 × 60 = 120 minutes.",
    "start_date": "22-09-2025T00:00:00",
    "end_date": "26-09-2025T23:59:59",
    "duration_minutes": 120
}}
"""

    human_message = """## Processing Instructions

**Current Datetime:** {request_datetime_str}
**Current Day:** {day}
**Email:** "{email_content}"

Analyze the email content using the current datetime as reference and return only the JSON output with your reasoning in the chain_of_thought field.

Output:
"""

    from langchain_core.prompts import ChatPromptTemplate

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", human_message)
    ])

    model = ChatOpenAI(
        model="Qwen/Qwen3-4B ",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
        api_key="abc-123",  # if you prefer to pass api key in directly instaed of using env vars
        base_url="http://localhost:4000/v1/",

        # organization="...",
        # other params...
        
    )

    chain = prompt | model | JsonOutputParser()
    try:
        response = chain.invoke({
            "request_datetime_str" : request_datetime_str,
            "day" : day,
            "email_content": email_content
        })

        summary = model.invoke(f'Generate the short professional summary for this content: {email_content}. Just return the header without any preamble').content
        return response, summary

    except (json.JSONDecodeError, AttributeError, KeyError, ValueError) as e:
        print(f"Error processing LLM response: {e}")
        return {
            'chain_of_thought': "Current date: 19-07-2025 (Friday). 'Today' = current date. No specific time mentioned, so applying defaults: start = 12:34:55 (current time), end = 23:59:59. Duration specified: 30 minutes.",
            'start_date': '19-07-2025T12:34:55',
            'end_date': '19-07-2025T23:59:59',
            'duration_minutes': 30
        }

In [8]:
extract_meeting_details_with_llm("Hi team, let's meet on next Thursday the **date is 24/07** start time is 00:00 and end time is 23:59 for 30 minutes to discuss the status of Agentic AI Project.", "19-07-2025T12:34:55")

({'chain_of_thought': "Current date: 19-07-2025 (Friday). 'Next Thursday' = current date + 7 days = 26-07-2025 (Thursday). However, the email explicitly mentions the date as 24/07, which is incorrect. Assuming the date is actually 24-07-2025 (Thursday) as per the email. Time conversion: start = 00:00:00, end = 23:59:59. Duration specified: 30 minutes. Using the explicitly mentioned date instead of the calculated date.",
  'start_date': '24-07-2025T00:00:00',
  'end_date': '24-07-2025T23:59:59',
  'duration_minutes': 30},
 'Team Meeting: Agentic AI Project Status Update')

In [18]:
import dateparser

print(dateparser.parse('next thursday', settings={'RELATIVE_BASE': datetime(2025, 7, 19)}))

None


In [17]:
import dateparser

date = dateparser.parse("March 15, 2021")
print(date)  # Output: 2021-03-15 00:00:00

2021-03-15 00:00:00


In [103]:
# The main scheduling function
from copy import deepcopy

def find_available_slot(request_data):
    """
    Finds the earliest available time slot for all attendees using an LLM for parsing.
    """
    # 1. Use LLM to extract meeting duration and target date
    details,summary = extract_meeting_details_with_llm(
        request_data["EmailContent"], 
        request_data["Datetime"]
    )
    print('LLM REsponse', details)

    ist_tz = timezone(timedelta(hours=5, minutes=30))
    start_date, end_date, duration_minutes = details['start_date'], details['end_date'], details['duration_minutes']
    meeting_duration = timedelta(minutes=duration_minutes)

    # 2. Determine the search window for the meeting
    start_dt = datetime.strptime(start_date, "%d-%m-%YT%H:%M:%S")
    end_dt = datetime.strptime(end_date, "%d-%m-%YT%H:%M:%S")

    day_start_utc = start_dt.replace(tzinfo=ist_tz)
    day_end_utc = end_dt.replace(tzinfo=ist_tz)

    print(day_start_utc, day_end_utc)
    # 3. Gather all attendees and their merged busy schedules
    all_emails = [request_data["From"]] + [p["email"] for p in request_data["Attendees"]]
    all_busy_slots = []
    personal_slots = {}
    for email in all_emails:
        calendar_events = retrive_calendar_events(
            email, 
            day_start_utc.isoformat(), 
            day_end_utc.isoformat()
        )
        personal_slots[email] = calendar_events
        all_busy_slots.extend(calendar_events)

    all_busy_slots.sort(key = lambda x: datetime.fromisoformat(x['StartTime']).astimezone(ist_tz))

    if not all_busy_slots:
        # If everyone is free, schedule it at the determined start time
        proposed_start = start_dt
        proposed_end = proposed_start + meeting_duration

        output_format = deepcopy(request_data)
        output_format["Attendees"] = [{
            "email": key,
            "events": value
        }
        for key,value in personal_slots.items()]

        output_format['EventStart'] = proposed_start.isoformat()
        output_format['EventEnd'] = proposed_end.isoformat()
        output_format['Duration_mins'] = duration_minutes
        output_format['metadata'] = {}

        for item in output_format['Attendees']:
            item["events"].append({
                'StartTime': output_format['EventStart'],
                'EndTime': output_format['EventEnd'],
                'NumAttendees': len(output_format["Attendees"]),
                'Attendees': list(personal_slots.keys()),
                'Summary': summary
            })
        return output_format
        
    # Merge overlapping intervals
    merged_busy_slots = [(datetime.fromisoformat(all_busy_slots[0]['StartTime']).astimezone(ist_tz),datetime.fromisoformat(all_busy_slots[0]['EndTime']).astimezone(ist_tz))]
    for event in all_busy_slots[1:]:
        current_start, current_end = datetime.fromisoformat(event['StartTime']).astimezone(ist_tz),datetime.fromisoformat(event['EndTime']).astimezone(ist_tz)
        last_start, last_end = merged_busy_slots[-1]
        if current_start < last_end:
            merged_busy_slots[-1] = (last_start, max(last_end, current_end))
        else:
            merged_busy_slots.append((current_start, current_end))
            
    # 4. Search for an available slot starting from the calculated search_start_time
    # Ensure our initial search time isn't already inside a busy slot
    if day_start_utc + meeting_duration <= merged_busy_slots[0][0]:
        proposed_start = start_dt
        proposed_end = proposed_start + meeting_duration

        output_format = deepcopy(request_data)
        output_format["Attendees"] = [{
            "email": key,
            "events": value
        }
        for key,value in personal_slots.items()]
            
        output_format['EventStart'] = proposed_start.isoformat()
        output_format['EventEnd'] = proposed_end.isoformat()
        output_format['Duration_mins'] = duration_minutes
        output_format['metadata'] = {}

        for item in output_format['Attendees']:
            item["events"].append({
                'StartTime': output_format['EventStart'],
                'EndTime': output_format['EventEnd'],
                'NumAttendees': len(output_format["Attendees"]),
                'Attendees': list(personal_slots.keys()),
                'Summary': summary
            })
        return output_format

    search_start_time = day_start_utc
    for busy_start, busy_end in merged_busy_slots:
        if search_start_time >= busy_start:
             search_start_time = max(search_start_time, busy_end)
             continue
        
        # Check if the gap between our pointer and the next busy slot is long enough
        gap_duration = busy_start - search_start_time
        if gap_duration >= meeting_duration:
            proposed_start = search_start_time
            proposed_end = proposed_start + meeting_duration
    
            output_format = deepcopy(request_data)
            output_format["Attendees"] = [{
                "email": key,
                "events": value
            }
            for key,value in personal_slots.items()]
                
            output_format['EventStart'] = proposed_start.isoformat()
            output_format['EventEnd'] = proposed_end.isoformat()
            output_format['Duration_mins'] = duration_minutes
            output_format['metadata'] = {}

            for item in output_format['Attendees']:
                item["events"].append({
                    'StartTime': output_format['EventStart'],
                    'EndTime': output_format['EventEnd'],
                    'NumAttendees': len(output_format["Attendees"]),
                    'Attendees': list(personal_slots.keys()),
                    'Summary': summary
                })
            return output_format
        
        # Not enough space, move our pointer to the end of this busy slot
        search_start_time = busy_end
        
    output_format = deepcopy(request_data)
    output_format["Attendees"] = [{
        "email": key,
        "events": value
    }
    for key,value in personal_slots.items()]
        
    output_format['EventStart'] = ""
    output_format['EventEnd'] = ""
    output_format['Duration_mins'] = duration_minutes
    output_format['metadata'] = {}
    return output_format


In [104]:
request1= {
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
    "Datetime": "19-07-2025T12:34:55",
    "Location": "IISc Bangalore",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Agentic AI Project Status Update",
    "EmailContent": "Hi team, let's meet on next Thursday the **date is 24/07** start time is 00:00 and end time is 23:59 for 30 minutes to discuss the status of Agentic AI Project."
}

print("--- Test Case 1: 'next week' ---")
find_available_slot(request1)

--- Test Case 1: 'next week' ---
LLM REsponse {'chain_of_thought': "Current date: 19-07-2025 (Friday). 'Next Thursday' = current date + 7 days = 26-07-2025 (Thursday). However, the email explicitly mentions the date as 24/07, which is incorrect. Assuming the date is actually 24-07-2025 (Thursday) as per the email. Time conversion: start = 00:00:00, end = 23:59:59. Duration specified: 30 minutes. Using the explicitly mentioned date instead of the calculated date.", 'start_date': '24-07-2025T00:00:00', 'end_date': '24-07-2025T23:59:59', 'duration_minutes': 30}
2025-07-24 00:00:00+05:30 2025-07-24 23:59:59+05:30
No of exceptions are:  0
No of exceptions are:  0
No of exceptions are:  0


{'Request_id': '6118b54f-907b-4451-8d48-dd13d76033a5',
 'Datetime': '19-07-2025T12:34:55',
 'Location': 'IISc Bangalore',
 'From': 'userone.amd@gmail.com',
 'Attendees': [{'email': 'userone.amd@gmail.com',
   'events': [{'StartTime': '2025-07-23T18:00:00+05:30',
     'EndTime': '2025-07-24T09:00:00+05:30',
     'NumAttendees': 1,
     'Attendees': ['SELF'],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-24T09:30:00+05:30',
     'EndTime': '2025-07-24T10:00:00+05:30',
     'NumAttendees': 3,
     'Attendees': ['userone.amd@gmail.com',
      'usertwo.amd@gmail.com',
      'userthree.amd@gmail.com'],
     'Summary': 'Agentic AI Project Status Update'},
    {'StartTime': '2025-07-24T18:00:00+05:30',
     'EndTime': '2025-07-25T09:00:00+05:30',
     'NumAttendees': 1,
     'Attendees': ['SELF'],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-24T09:00:00+05:30',
     'EndTime': '2025-07-24T09:30:00+05:30',
     'NumAttendees': 3,
     'Attendees': ['userone.amd@gmail.com'

In [ ]:
request_thursday = {
    "Request_id": "a2", "Datetime": "19-07-2025T12:34:55", "From": "userone.amd@gmail.com",
    "Attendees": [{"email": "usertwo.amd@gmail.com"}, {"email": "userthree.amd@gmail.com"}],
    "EmailContent": "Let's sync up on Thursday for half an hour."
}

print("\n\n--- Test Case 2: 'Thursday' ---")
start_time_2, end_time_2 = find_meeting_slot(request_thursday)
if start_time_2 and end_time_2:
    print(f"\nFound an available slot!")
    print(f"  Start Time: {start_time_2}")
    print(f"  End Time:   {end_time_2}")
else:
    print("\nCould not find a suitable meeting time in the requested window.")
